In [1]:
import Pkg; Pkg.activate("..")

  Activating project at `~/SAFT_ML`


In [2]:
using CSV, DataFrames, DelimitedFiles, Clapeyron, PyCall
import PyPlot; const plt = PyPlot ;

In [3]:
# Open and structure Clapetron SAFTVR-Mie data
# Set missing values to -1 for easier processing (all valid values are positive)

raw_data_saft_vr_mie = CSV.read("SAFTVRMie_like.csv",DataFrame,header=3) 

raw_data_saft_vr_mie[:,:source] = replace(raw_data_saft_vr_mie[:,:source], missing => "-") ;
raw_data_saft_vr_mie = filter(row -> row.species != "n-eicosane", raw_data_saft_vr_mie)
raw_data_saft_vr_mie = filter(row -> row.species != "n-dodecane", raw_data_saft_vr_mie)
raw_data_saft_vr_mie = filter(row -> row.species != "ethanol", raw_data_saft_vr_mie)

raw_data_saft_vr_mie.n_H = Float64.(raw_data_saft_vr_mie.n_H)
raw_data_saft_vr_mie.n_e = Float64.(raw_data_saft_vr_mie.n_e)
raw_data_saft_vr_mie.source = String31.(raw_data_saft_vr_mie.source) 
raw_data_saft_vr_mie.species = String.(raw_data_saft_vr_mie.species) ;

raw_data_saft_vr_mie ;

In [4]:
num_rows = nrow(raw_data_saft_vr_mie)

validation_data_saft_vr_mie = DataFrame(
    species = fill(missing, num_rows),
    Mw = fill(0.0, num_rows),
    source = fill(missing, num_rows),
    segment = fill(0.0, num_rows),
    sigma = fill(0.0, num_rows),
    epsilon = fill(0.0, num_rows),
    lambda_r = fill(0.0, num_rows),
    lambda_a = fill(0.0, num_rows),
    n_H = fill(0.0, num_rows),
    n_e = fill(0.0, num_rows),

    crit_t_K_clapeyron = fill(0.0,num_rows),
    crit_p_MPa_clapeyron = fill(0.0,num_rows),
    crit_v_m3_per_mol_clapeyron = fill(0.0,num_rows),
    sat_t_K_clapeyron = fill([0.0],num_rows),
    sat_p_MPa_clapeyron = fill([0.0],num_rows),
    sat_vliq_m3_per_mol_clapeyron = fill([0.0],num_rows),
    sat_vvap_m3_per_mol_clapeyron = fill([0.0],num_rows),

    crit_t_K_python = fill(0.0,num_rows),
    crit_p_MPa_python = fill(0.0,num_rows),
    crit_v_m3_per_mol_python = fill(0.0,num_rows),
    sat_t_K_python = fill([0.0],num_rows),
    sat_p_MPa_python = fill([0.0],num_rows),
    sat_vliq_m3_per_mol_python = fill([0.0],num_rows),
    sat_vvap_m3_per_mol_python = fill([0.0],num_rows),

    sat_t_K_AAD = fill(0.0,num_rows),
    sat_p_MPa_AAD = fill(0.0,num_rows),
    sat_vliq_m3_per_mol_AAD = fill(0.0,num_rows),
    sat_vvap_m3_per_mol_AAD = fill(0.0,num_rows),
) ;

In [5]:
validation_data_saft_vr_mie.species = raw_data_saft_vr_mie.species
validation_data_saft_vr_mie.Mw = raw_data_saft_vr_mie.Mw
validation_data_saft_vr_mie.source = raw_data_saft_vr_mie.source
validation_data_saft_vr_mie.segment = raw_data_saft_vr_mie.segment
validation_data_saft_vr_mie.sigma = raw_data_saft_vr_mie.sigma
validation_data_saft_vr_mie.epsilon = raw_data_saft_vr_mie.epsilon
validation_data_saft_vr_mie.lambda_r = raw_data_saft_vr_mie.lambda_r
validation_data_saft_vr_mie.lambda_a = raw_data_saft_vr_mie.lambda_a
validation_data_saft_vr_mie.n_H = raw_data_saft_vr_mie.n_H
validation_data_saft_vr_mie.n_e = raw_data_saft_vr_mie.n_e

species_names = validation_data_saft_vr_mie.species ;

# validation_data_saft_vr_mie ;

In [11]:
model = SAFTVRMie(["ethanol"]) ;
critical_props = crit_pure(model)

# # temp_range = collect(range(0.5*critical_props[1],critical_props[1],50)) ;
# # for i = 1:50
# #     print(i)
# #     sat_props = [saturation_pressure(model,temp_range[i])] ;
# # end

(336.2999999999998, -6.618332568142466e7, 8.767649180068293e-5)

In [9]:
# This cell generates data for all ~1800 species in Esper et al.
num_rows = nrow(validation_data_saft_vr_mie)

n = 500

for i in species_names

    row_number = findall(validation_data_saft_vr_mie.species .== i)[1]
    println(row_number," ",i)

    model = SAFTVRMie(i) ;
    critical_props = crit_pure(model)

    temp_range = collect(range(0.5*critical_props[1],critical_props[1],n)) ;
    sat_props = [saturation_pressure(model,temp_range[i]) for i ∈ 1:n] ;

    sat_pressures = [sat_props[i][1] for i ∈ 1:n] ;
    sat_vols_liq = [sat_props[i][2] for i ∈ 1:n] ;
    sat_vols_vap = [sat_props[i][3] for i ∈ 1:n] ;

    for i in 1:length(critical_props)
        validation_data_saft_vr_mie[row_number, 10 + i] = critical_props[i]
    end

    validation_data_saft_vr_mie[row_number,:sat_t_K_clapeyron] = temp_range
    validation_data_saft_vr_mie[row_number,:sat_p_MPa_clapeyron] = sat_pressures
    validation_data_saft_vr_mie[row_number,:sat_vliq_m3_per_mol_clapeyron] = sat_vols_liq
    validation_data_saft_vr_mie[row_number,:sat_vvap_m3_per_mol_clapeyron] = sat_vols_vap

end

1 methane
2 ethane
3 n-propane
4 n-butane
5 n-pentane
6 n-hexane
7 n-heptane
8 n-octane
9 n-nonane
10 n-decane
11 n-pentadecane
12 tetrafluoromethane
13 hexafluoroethane
14 octafluoropropane
15 decafluorobutane
16 n-perfluoropentane
17 n-perfluorohexane
18 fluorine
19 carbon dioxide
20 carbon dioxide_lafitte
21 benzene
22 toluene
23 water
24 methanol
25 propan-1-ol
26 butan-1-ol
27 helium-4


In [10]:
# Data storage
CSV.write("saft_vr_mie_validation_data.csv", validation_data_saft_vr_mie) ;

In [ ]:
# Bugs
# - Parameters for n-dodecane are incorrect in SAFT-VR-Mie "like" data file (https://spiral.imperial.ac.uk/bitstream/10044/1/12859/2/J%20Chem%20Phys%202013%20Lafitte-1%20copy.pdf)
    #  - Means critical pressure returns as NaN
# - Parameters missing for n-eicosane
# - Something wrong with ethanol parameters
    #  - Means critical pressure returns as negative